# The compact Genetic Algorithm (cGA)

This notebook contains a simple implementation of the compact Genetic Algoritm (cGA) for continous domains.

In [1]:
import evoflow.backend as B

# Constants.

LENGTH   = 5     # The length of the problem to solve.
POP_SIZE = 1000  # The population size equivalent.
EPOCHS   = 20    # Number of epochs to run.
EPSILON  = 0.1   # Epsilon tolerance per loci.

Using numpy backend


## Function to optimize: OneMax

The traditional OneMax function add the bits sets to one on a binary encoded individual.

![OneMax for binary individuals](./evoflow-OneMax-Discrete.jpg)

OneMax can be extended to work with individuals defined over real numbers instead. If you defined as before, there would be an assymptotic vaule that maximizes it, when all loci values are `Inf`. You can just tweak the definition to provide a single optima for maximization. There are multiple ways to do so, but the simplest one is to choose a goal an turn OneMax into a concave function, as shown before.

![OneMax for binary individuals](./evoflow-OneMax-Continuous.jpg)

A simple implementation of this idea for ag given goal where all loci values should be 3 is shown below.

In [2]:
goal = B.full(LENGTH, 3)

def one_max(samples):
    diff = B.subtract(goal, samples)
    return B.multiply(B.sum(B.multiply(diff, diff), axis=1), -1)

## cGA model

The initial cGA model is just a sequence of normal distribution params (_mean_ and _deviation_). The model is encoded into a two-dimensional tensor. One dimention has only two entries per locus, the mean and the deviation for that _ith_ position, whereas the other dimension has as many elements as LENGTH of the problem to solve.

In [3]:
def initial_model(length):
    return B.concatenate((B.normal(length)[:, None], B.full(length, 1)[:, None]), axis=1)

## A cGA iteration model update

A cGA iteration is prety straight forward. If follows these steps:

1. Sample the model generating SAMPLE_SIZE individuals
2. Compute their fitness.
3. Compute the winner.
4. Compute the model update.
5. Update the model.

It is important to note here that this simple model update does not anneal the standard deviation, since naive versions can lead to premature convergence.

In [4]:
def update_model(model, function, pop_size=POP_SIZE, length=LENGTH):
    samples = B.normal((2, length), mean=model[:, 0], dev=model[:, 1])
    fitness = function(samples)
    winner = B.top_k_indices(fitness, 1)
    losser = B.bottom_k_indices(fitness, 1)
    inc = B.subtract(samples[winner, :], samples[losser, :])  # Because the goal is to maximize the function.
    inc = B.divide(inc, pop_size)
    update = B.concatenate((inc, B.zeros((1, length))), axis=0).T
    return B.add(model, update)

model = initial_model(LENGTH)

print('Original')
print(model)

print('Updated')
print(update_model(model, one_max))
    

Original
[[ 0.73988103  1.        ]
 [-0.70295473  1.        ]
 [-0.89169912  1.        ]
 [-1.14859906  1.        ]
 [ 1.83488457  1.        ]]
Updated
[[ 0.73967083  1.        ]
 [-0.70407742  1.        ]
 [-0.89056885  1.        ]
 [-1.14793012  1.        ]
 [ 1.83468195  1.        ]]


## Evolving

At this point the only thing you need is to iterate till it converges. Since it may not, depending on the function you are trying to optimize, you can just run it for _N_ epochs. Each epoch requires _POP_SIZE_ updates. For simplicitly, the example below only shows the _mean_ part of the model since we are not annealing the _deviation_

In [5]:
model = initial_model(LENGTH)
print('Initial model')
print(model[:, 0])
print()
for e in range(EPOCHS):
    for i in range(POP_SIZE):
        model = update_model(model, one_max)
    print('EPOCH {:0>3}'.format(e), model[:, 0])

Initial model
[-0.12711348 -0.05263459  0.150466   -0.93838774 -1.6095832 ]

EPOCH 000 [ 0.32454931  0.39582391  0.5724013  -0.40794401 -1.00171096]
EPOCH 001 [ 0.68246754  0.7632961   0.88875995  0.09124844 -0.33678659]
EPOCH 002 [1.13065864 1.16483017 1.24944084 0.69672454 0.28556927]
EPOCH 003 [1.50786419 1.64988727 1.58867174 1.30242431 0.93053742]
EPOCH 004 [1.96224529 2.03072249 2.01235858 1.74151877 1.47431644]
EPOCH 005 [2.36226558 2.37382331 2.41124319 2.15616381 2.02829954]
EPOCH 006 [2.67547593 2.65908318 2.62255396 2.54856183 2.45109458]
EPOCH 007 [2.81920926 2.81170262 2.80047879 2.73459494 2.7059748 ]
EPOCH 008 [2.93517349 2.88468553 2.82846825 2.80110112 2.8134341 ]
EPOCH 009 [2.96876972 2.95786113 2.91587689 2.90678067 2.93334618]
EPOCH 010 [2.99051756 2.95031554 2.96398364 2.89760522 2.96105238]
EPOCH 011 [2.97593528 2.99701435 2.97712502 2.99728721 3.02363684]
EPOCH 012 [2.99005091 2.99284946 2.93406016 3.01132269 3.03378963]
EPOCH 013 [2.97802    3.02375338 2.976274 

Instead of epoch you con just run it till converges. Since we are optimizing on a continuous space, convergene is aproximize via _epsilon_, and defined as follows.

In [6]:
def converged(model, goal, epsilon=EPSILON):
    return B.allclose(goal, model[:,0], absolute_tolerance=epsilon)

With this definition, you can redefine the main evolutionary loop as follows.

In [7]:
model = initial_model(LENGTH)
print('Initial model')
print(model[:, 0])
print()
e = 0
while not converged(model, goal):
    for i in range(POP_SIZE):
        model = update_model(model, one_max)
    print('EPOCH {:0>3}'.format(e), model[:, 0])
    e += 1

Initial model
[-0.084807    1.01313895 -1.31013415 -1.09989419  1.39936609]

EPOCH 000 [ 0.39735516  1.31879292 -0.62010649 -0.46936052  1.62443461]
EPOCH 001 [ 0.82953033  1.70721744 -0.00642011  0.12970649  1.91716234]
EPOCH 002 [1.29564099 1.89756373 0.62588527 0.72377222 2.25356624]
EPOCH 003 [1.76051501 2.20684613 1.19455954 1.2928632  2.41856417]
EPOCH 004 [2.14209331 2.43864782 1.7686297  1.82233999 2.61581894]
EPOCH 005 [2.47547288 2.62196709 2.19463911 2.33288178 2.74525052]
EPOCH 006 [2.71662897 2.82139654 2.5277133  2.60635835 2.88149516]
EPOCH 007 [2.90216973 2.89095892 2.74801927 2.84944784 2.93498258]
EPOCH 008 [2.93638667 2.93761511 2.9057087  2.96736852 2.93568425]
